# LazyArray UDF DSL Kernels

`@blosc2.dsl_kernel` lets you write kernels with Python function syntax while executing through the miniexpr DSL path.

Use DSL kernels when you want:

- A vectorized UDF model (operate over NDArray chunks/blocks, not Python scalar loops)
- Optional JIT compilation via miniexpr backends (for example `tcc`/`cc`) without requiring Numba
- Early syntax validation and actionable diagnostics for unsupported constructs

This tutorial complements `03.lazyarray-udf.ipynb` (generic Python UDFs).

For the canonical DSL syntax contract, see the miniexpr docs: `doc/dsl-syntax.md`.


In [1]:
import numpy as np

import blosc2

## 1. Define a DSL Kernel

A valid DSL kernel can be used with `blosc2.lazyudf(...)` like a regular UDF.

In [2]:
@blosc2.dsl_kernel
def kernel_index_ramp(x):
    # _i* and _n* are reserved DSL index/shape symbols, so disable linter warnings
    return x + _i0 * _n1 + _i1  # noqa: F821

In [3]:
shape = (5, 10)
x = blosc2.ones(shape, dtype=np.float32)
expr = blosc2.lazyudf(kernel_index_ramp, (x,), dtype=np.float32)
res = expr[:]
res

array([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
       [11., 12., 13., 14., 15., 16., 17., 18., 19., 20.],
       [21., 22., 23., 24., 25., 26., 27., 28., 29., 30.],
       [31., 32., 33., 34., 35., 36., 37., 38., 39., 40.],
       [41., 42., 43., 44., 45., 46., 47., 48., 49., 50.]], dtype=float32)

In [4]:
# Optional: request miniexpr JIT backend for this DSL kernel
expr_jit = blosc2.lazyudf(
    kernel_index_ramp,
    (x,),
    dtype=x.dtype,
    jit=True,
    jit_backend="tcc",
)
res_jit = expr_jit.compute()
res_jit[:2, :5]

array([[ 1.,  2.,  3.,  4.,  5.],
       [11., 12., 13., 14., 15.]], dtype=float32)

## 2. Preflight Validation (`validate_dsl`)

You can validate a kernel and inspect diagnostics without executing it.

In [5]:
report_ok = blosc2.validate_dsl(kernel_index_ramp)
report_ok

{'valid': True,
 'dsl_source': 'def kernel_index_ramp(x):\n    # _i* and _n* are reserved DSL index/shape symbols, so disable linter warnings\n    return x + _i0 * _n1 + _i1  # noqa: F821',
 'input_names': ['x'],
 'error': None}

## 3. Invalid Syntax Example

Python ternary expressions are not part of the DSL subset.
`validate_dsl` reports the issue, and `lazyudf(...)` raises early with a detailed message.

In [6]:
@blosc2.dsl_kernel
def kernel_invalid_ternary(x):
    return 1 if x else 0


report_bad = blosc2.validate_dsl(kernel_invalid_ternary)
print(report_bad["valid"])
print(report_bad["error"])

False
Ternary expressions are not supported in DSL; use where(cond, a, b) at line 2, column 14

DSL kernel source:
1 | def kernel_invalid_ternary(x):
2 |     return 1 if x else 0
  |              ^

See: https://github.com/Blosc/miniexpr/blob/main/doc/dsl-usage.md


## 4. Advanced Example: Mandelbrot DSL

For a more advanced real-world DSL kernel, see:

- `examples/ndarray/mandelbrot-dsl.ipynb`

GitHub link:

- https://github.com/Blosc/python-blosc2/blob/main/examples/ndarray/mandelbrot-dsl.ipynb